In [ ]:
PROMPT = """
Bạn là một trợ lý AI có nhiệm vụ mô tả một người trong hình ảnh. Hãy tạo một chú thích chi tiết theo các hướng dẫn sau.

Hãy quan sát kỹ hình ảnh bên dưới và mô tả lại một cách chi tiết, đầy đủ và khách quan bằng tiếng Việt. Miêu tả rõ:
– Những sự vật, con người, động vật… có mặt trong ảnh.
– Hành động, tư thế, tương tác giữa các đối tượng nếu có.
– Màu sắc, trang phục, biểu cảm gương mặt, ánh sáng, thời gian (nếu đoán được).
– Bối cảnh xung quanh: địa điểm, không gian (trong nhà/ngoài trời), thời tiết, môi trường…
– Bất kỳ chi tiết nào nổi bật hoặc thú vị trong ảnh.

Không thêm suy đoán không có cơ sở. Chỉ mô tả những gì nhìn thấy rõ trong ảnh.

## Ví dụ:

Dưới đây là một chú thích mẫu:
Ví dụ 1: "Một người đàn ông trung niên đang đứng trên vỉa hè vào ban ngày. Anh mặc áo sơ mi trắng, quần âu tối màu và đeo kính râm. Gương mặt nghiêm túc, ánh mắt nhìn về phía trước. Phía sau là một dãy nhà phố và vài chiếc xe hơi đang đỗ bên đường"
Ví dụ 2: "Một con phố trung tâm đông đúc vào giờ tan tầm. Hàng chục người đang đi bộ vội vã trên vỉa hè, nhiều người mặc áo sơ mi, đeo balo hoặc cầm điện thoại. Các phương tiện chen chúc nhau trên lòng đường, còi xe vang lên inh ỏi. Các biển hiệu quảng cáo sáng rực và cửa hàng hai bên tấp nập khách ra vào"
Ví dụ 3: "Bên trong một khu chợ truyền thống đông đúc, người bán và người mua chen nhau trong không gian chật hẹp. Các sạp hàng đầy ắp rau củ, thịt cá, và đồ khô được bày ra san sát nhau. Tiếng rao bán, tiếng trả giá vang lên hỗn loạn. Không khí náo nhiệt và có phần oi bức, ánh sáng lờ mờ từ các bóng đèn treo lủng lẳng phía trên"

Hãy sử dụng ví dụ này để tham khảo về phong cách và cấu trúc, nhưng **không sao chép các chi tiết cụ thể**. **Bỏ qua các yếu tố bất thường trong ví dụ nếu chúng không liên quan**. Luôn mô tả người trong ảnh một cách **trung thực và toàn diện**.

Chỉ trả về câu caption hoàn chỉnh không thêm hướng dẫn gì thêm.
"""

In [ ]:
def generate_content(client, model_name, image_url, prompt, temperature=0.7, top_p=0.9):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        },
                    },
                ],
            }
        ],
        temperature=temperature,
        top_p=top_p,
    )

    return response.choices[0].message.content

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="YOUR_API_KEY", base_url="http://0.0.0.0:8000/v1")
model_name = client.models.list().data[0].id
model_name

In [ ]:
from pathlib import Path

def list_sorted_images(folder):
    # Define common image extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}

    folder_path = Path(folder)
    # Get all image files in folder and subfolders
    image_files = [f for f in folder_path.rglob('*') if f.suffix.lower() in image_extensions]

    # Sort the files (by path)
    sorted_images = sorted(image_files)

    # Return as list of strings (paths)
    return [str(f) for f in sorted_images]

def gen_image_url(file):
    return f"http://0.0.0.0:8001/{file}"

In [ ]:
# import os
# import unicodedata

# def strip_accents_and_spaces(text):
#     """
#     Remove accents and spaces from Vietnamese names.
#     E.g., "Việt Nam file.txt" → "VietNamfile.txt"
#     """
#     # Remove accents
#     text = unicodedata.normalize('NFD', text)
#     text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
#     # Remove all spaces
#     text = text.replace(' ', '')
#     return text

# def rename_all(root_path):
#     # Rename files and folders bottom-up to avoid path issues
#     for current_path, dirnames, filenames in os.walk(root_path, topdown=False):
#         # Rename files
#         for filename in filenames:
#             old_file_path = os.path.join(current_path, filename)
#             new_file_name = strip_accents_and_spaces(filename)
#             new_file_path = os.path.join(current_path, new_file_name)
#             if old_file_path != new_file_path:
#                 print(f"Renaming file: {old_file_path} → {new_file_path}")
#                 os.rename(old_file_path, new_file_path)

#         # Rename subfolders
#         for dirname in dirnames:
#             old_dir_path = os.path.join(current_path, dirname)
#             new_dir_name = strip_accents_and_spaces(dirname)
#             new_dir_path = os.path.join(current_path, new_dir_name)
#             if old_dir_path != new_dir_path:
#                 print(f"Renaming folder: {old_dir_path} → {new_dir_path}")
#                 os.rename(old_dir_path, new_dir_path)

#     # Rename root folder last
#     parent_dir = os.path.dirname(root_path)
#     root_name = os.path.basename(root_path)
#     new_root_name = strip_accents_and_spaces(root_name)
#     new_root_path = os.path.join(parent_dir, new_root_name)
#     if root_path != new_root_path:
#         print(f"Renaming root folder: {root_path} → {new_root_path}")
#         os.rename(root_path, new_root_path)

# rename_all('HINH ANH')

In [ ]:
# !pip3 install 'numpy<2'

In [ ]:
import os
import sys
import random
import time
from collections import defaultdict

import cv2
import matplotlib.pyplot as plt

files = defaultdict(list)
for file in list_sorted_images('HINHANH'):
    if '.ipynb_checkpoints' in file:
        continue
    files[file.split('/')[1]].append(file)

for image_type, files in files.items():
    for file in random.choices(files, k=3):
        print(file)
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.axis('off')
        plt.show()

        start_time = time.time()
        caption = generate_content(client, model_name, gen_image_url(file), PROMPT, temperature=0.6, top_p=0.8)
        end_time = time.time()
        print(caption)
        print(end_time - start_time)